### Length_penalty

In [13]:
def length_penalty_answer(pred, truth, ratio=1.3):
    pred_len = len(pred.split())
    truth_len = len(truth.split())
    max_ok = round(truth_len * ratio) # mở rộng biên độ chấp nhận
    
    if pred_len <= max_ok:
        return 1.0
    else:
        excess_ratio = (pred_len - max_ok) / truth_len
        return - min(excess_ratio, 1.0)

def length_penalty_explanation(pred, truth, ratio=1.2):
    pred_len = len(pred.split())
    truth_len = len(truth.split())
    min_ok = max(5, int(truth_len * 0.7)) # mở rộng biên độ chấp nhận
    max_ok = round(truth_len * ratio) # mở rộng biên độ chấp nhận
    
    if min_ok <= pred_len <= max_ok:
        return 1.0
    elif pred_len < min_ok:
        return -0.5
    else:
        excess_ratio = (pred_len - max_ok) / truth_len
        return - min(excess_ratio, 1.0)

In [14]:
print(length_penalty_answer("Đỏ", "Đỏ"))      
print(length_penalty_answer("Đỏ", "Màu đỏ"))  
print(length_penalty_answer("Màu đỏ", "Đỏ"))  
print(length_penalty_answer("Màu đỏ sáng", "Màu đỏ"))  
print(length_penalty_answer("Đỏ", "Màu đỏ tươi"))  
print(length_penalty_answer("Màu đỏ tươi rực", "Màu đỏ tươi"))  
print(length_penalty_answer("Màu đỏ tươi rực rỡ", "Màu đỏ tươi"))  

1.0
1.0
-1.0
1.0
1.0
1.0
-0.3333333333333333


In [15]:
print(length_penalty_explanation("Rất rõ ràng bức ảnh có hai con chó màu đen lớn đang đứng cạnh nhau", "Ảnh có hai con chó")) 
print(length_penalty_explanation("Bức ảnh này có hai con chó màu đen ở giữa công viên vào buổi sáng rất đẹp", "Hai con chó màu đen")) 
print(length_penalty_explanation("Có hai con chó", "Có hai con chó màu đen đang ở giữa ảnh vào buổi sáng")) 
print(length_penalty_explanation("Hai con chó ở giữa", "Có hai con chó màu đen đang ở giữa ảnh vào buổi sáng")) 
print(length_penalty_explanation("Có hai con chó màu đen", "Có hai con chó màu đen đang ở giữa ảnh vào buổi sáng")) 
print(length_penalty_explanation("Có hai con chó màu đen đang ở trên cỏ công viên", "Có hai con chó màu đen đang ở giữa ảnh vào buổi sáng")) 
print(length_penalty_explanation("Hai con chó", "Có hai con chó màu đen đang ở giữa ảnh vào buổi sáng")) 


-1.0
-1.0
-0.5
-0.5
-0.5
1.0
-0.5


In [1]:
def length_penalty_explanation(pred, truth, ratio=1.2, sentence_penalty_weight=0.3):
    """
    Penalty function với kiểm soát số câu:
    - Nếu pred có >= 2 câu (phân tách bởi '.' hoặc ',') -> phạt dựa trên số câu
    - Nếu pred chỉ 1 câu -> kiểm tra độ dài như bình thường
    
    Args:
        pred: Câu prediction
        truth: Câu ground truth
        ratio: Tỷ lệ độ dài tối đa cho phép
        sentence_penalty_weight: Trọng số phạt cho mỗi câu thừa (mặc định 0.3)
    
    Returns:
        float: 1.0 (ok), -0.5 (quá ngắn), hoặc giá trị âm (quá dài/nhiều câu)
    """
    # Tách câu dựa trên dấu '.' hoặc ',' (loại bỏ câu rỗng)
    import re
    pred_sentences = [s.strip() for s in re.split(r'[.,]', pred) if s.strip()]
    truth_sentences = [s.strip() for s in re.split(r'[.,]', truth) if s.strip()]
    
    num_pred_sentences = len(pred_sentences)
    num_truth_sentences = len(truth_sentences)
    
    # Kiểm tra số câu: Nếu pred có >= 2 câu mà truth chỉ 1 câu -> phạt
    if num_truth_sentences == 1 and num_pred_sentences >= 2:
        # Phạt càng nặng khi gen càng nhiều câu
        # Ví dụ: 2 câu -> -0.3, 3 câu -> -0.6, 4 câu -> -0.9, 5+ câu -> -1.0
        excess_sentences = num_pred_sentences - 1
        penalty = -min(excess_sentences * sentence_penalty_weight, 1.0)
        return penalty
    
    # Nếu số câu hợp lệ -> kiểm tra độ dài từ
    pred_len = len(pred.split())
    truth_len = len(truth.split())
    min_ok = max(5, int(truth_len * 0.7))
    max_ok = round(truth_len * ratio)
    
    if min_ok <= pred_len <= max_ok:
        return 1.0
    elif pred_len < min_ok:
        return -0.5
    else:
        excess_ratio = (pred_len - max_ok) / truth_len
        return -min(excess_ratio, 1.0)


In [ ]:
truth = "Đây là một câu ground truth."

# Case 1: Predict 1 câu, độ dài OK
pred1 = "Đây là câu trả lời phù hợp với độ dài."
print(f"1 câu OK: {length_penalty_explanation(pred1, truth)}") -.05

# Case 2: Predict 2 câu -> bị phạt
pred2 = "Đây là câu đầu tiên. Đây là câu thứ hai."
print(f"2 câu: {length_penalty_explanation(pred2, truth)}")  # -0.3

# Case 3: Predict 3 câu -> phạt nặng hơn
pred3 = "Câu một. Câu hai. Câu ba."
print(f"3 câu: {length_penalty_explanation(pred3, truth)}")  # -0.6

# Case 4: Predict 5 câu -> phạt tối đa
pred4 = "Câu 1. Câu 2. Câu 3. Câu 4. Câu 5."
print(f"5 câu: {length_penalty_explanation(pred4, truth)}")  # -1.0

# Case 5: Predict 1 câu nhưng quá ngắn
pred5 = "Ngắn quá."
print(f"1 câu ngắn: {length_penalty_explanation(pred5, truth)}")  # -0.5

1 câu OK: -0.5
2 câu: -0.3
3 câu: -0.6
5 câu: -1.0
1 câu ngắn: -0.5


### Rough L

In [16]:
from pycocoevalcap.rouge.rouge import Rouge

def rouge_l_batch_reward(predictions_list, ground_truths_list):
    gts = {}
    res = {}
    
    for idx, (pred, truth) in enumerate(zip(predictions_list, ground_truths_list)):
        sample_id = f'sample_{idx}'
        gts[sample_id] = [truth.lower().strip()]
        res[sample_id] = [pred.lower().strip()]
    
    scorer = Rouge()
    avg_score, individual_scores = scorer.compute_score(gts, res)
    return {
        'avg_score': avg_score,
        'individual_scores': individual_scores
    }


# Ví dụ
predictions = [
    "Trượt Ván",
    "Cuộn quay",
    "Trượt sóng",
]

ground_truths = [
    "lướt sóng",
    "lướt sóng",
    "lướt sóng",
]

results = rouge_l_batch_reward(predictions, ground_truths)
print(f"Average ROUGE-L: {results['avg_score']:.4f}")
print(f"Individual scores: {results['individual_scores']}")


Average ROUGE-L: 0.1667
Individual scores: [0.  0.  0.5]


In [17]:
import torch
from torchmetrics.text import BERTScore

# 1. Khởi tạo BERTScore với PhoBERT
device = "cuda" if torch.cuda.is_available() else "cpu"
phobert_scorer = BERTScore(
    model_name_or_path="/mnt/dataset1/pretrained_fm/vinai/phobert-base",
    num_layers=12,
    rescale_with_baseline=False,
    device=device
)

In [18]:
predictions = [
    "có 3 con mèo",
    "Cuộn quay",
    "Trượt sóng",
    "ô tô",
    "chú chó AKITA"
]

ground_truths = [
    "có 5 con mèo",
    "lướt sóng",
    "lướt sóng",
    "xe hơi",
    "chó nhật bản"
]

In [19]:
# Tính BERTScore
phobert_scorer.reset() 
phobert_scorer.update(predictions, ground_truths)
scores = phobert_scorer.compute()

# Lấy kết quả (precision, recall, f1)
f1_scores = scores['f1'].tolist()  # [0.85, 0.78] (ví dụ)
print(f"F1 Scores: {f1_scores}")

The following layers were not sharded: encoder.layer.*.attention.self.query.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, embeddings.position_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.query.bias, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, embeddings.LayerNorm.bias, embeddings.token_type_embeddings.weight, pooler.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.attention.self.key.bias, embeddings.word_embeddings.weight, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value.bias, encoder.layer.*.attention.output.dense.weight, pooler.dense.bias, encoder.layer.*.attention.self.value.weight, encoder.layer.*.attention.self.key.weight


F1 Scores: [0.20858220756053925, 0.23071549832820892, 0.22716161608695984, 0.31682369112968445, 0.2283453345298767]


In [20]:
f1_scores = []
for pred, ref in zip(predictions, ground_truths):
    phobert_scorer.reset()
    phobert_scorer.update([pred], [ref])
    score = phobert_scorer.compute()
    f1_scores.append(score['f1'].item())


print(f"F1 Scores: {f1_scores}")

The following layers were not sharded: encoder.layer.*.attention.self.query.weight, encoder.layer.*.intermediate.dense.bias, encoder.layer.*.attention.output.dense.bias, encoder.layer.*.output.LayerNorm.bias, encoder.layer.*.attention.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, embeddings.position_embeddings.weight, encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.query.bias, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, embeddings.LayerNorm.bias, embeddings.token_type_embeddings.weight, pooler.dense.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.attention.self.key.bias, embeddings.word_embeddings.weight, embeddings.LayerNorm.weight, encoder.layer.*.attention.self.value.bias, encoder.layer.*.attention.output.dense.weight, pooler.dense.bias, encoder.layer.*.attention.self.value.weight, encoder.layer.*.attention.self.key.weight
The following layers were not sharded: encoder.layer.*.attentio

F1 Scores: [0.8428176641464233, 0.345912367105484, 0.5009145140647888, 0.2785395681858063, 0.4424190819263458]
